Next_word_prediction_lstm


In [36]:
##Data Cleaning and Preprocessing

import nltk
nltk.download("gutenberg")
from nltk.corpus import gutenberg

## load the dataset
data=gutenberg.raw("shakespeare-hamlet.txt")

##save the file
with open ("hamlet.txt","w") as file :
  file.write(data)

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [37]:
### Data Preprocessing
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Load the dataset
with open("hamlet.txt", "r") as file:
    text = file.read().lower()  # Convert the text to lowercase

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])  # Passing the entire text as a list

# Calculate total words (size of the vocabulary)
total_words = len(tokenizer.word_index) + 1  # Add 1 for padding index

total_words  # Output the total number of unique words


4818

In [38]:
### Create Input Sequences
input_sequences = []

for line in text.split("\n"):
    # Tokenize each line into a list of tokens
    token_list = tokenizer.texts_to_sequences([line])[0]  # Corrected tokenization

    # Create n-gram sequences
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)  # Append each n-gram sequence

input_sequences  # Output the list of input sequences

###pad_sequences
max_sequence_len=max([len(x) for x in input_sequences])
max_sequence_len


input_sequences=np.array(pad_sequences(input_sequences,maxlen=max_sequence_len,padding="pre"))
input_sequences


array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]], dtype=int32)

In [39]:
###create predictors and labels

import tensorflow as tf

x,y=input_sequences[:,:-1],input_sequences[:,-1]

x
y

y = tf.keras.utils.to_categorical(y, num_classes=total_words)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [40]:
####split the data into training and testing

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [44]:
###define early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping=EarlyStopping(monitor="val_loss",patience=3,restore_best_weights=True)


In [41]:
########.................. TRAIN OUR LSTM MODEL...................... ######

In [42]:
## Train our LSTM RNN

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout,GRU

## Define the model
model=Sequential()
model.add(Embedding(total_words,100,input_length=max_sequence_len-1))
model.add(LSTM(150,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words,activation="softmax"))

# #Compile the model
model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_8 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_9 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [46]:
###train the model
history=model.fit(x_train,y_train,epochs=50,validation_data=(x_test,y_test),verbose=1,callbacks=[early_stopping])

Epoch 1/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 56s 83ms/step - accuracy: 0.0304 - loss: 7.1690 - val_accuracy: 0.0336 - val_loss: 6.6875
Epoch 2/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 79s 79ms/step - accuracy: 0.0359 - loss: 6.4710 - val_accuracy: 0.0420 - val_loss: 6.7467
Epoch 3/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 80s 76ms/step - accuracy: 0.0432 - loss: 6.3287 - val_accuracy: 0.0484 - val_loss: 6.7797
Epoch 4/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 85s 80ms/step - accuracy: 0.0550 - loss: 6.1559 - val_accuracy: 0.0499 - val_loss: 6.8093


In [49]:
### Function to predict the next word
def predict_new_word(model, tokenizer, text, max_sequence_len):
    token_list = tokenizer.texts_to_sequences([text])[0]
    if len(token_list) >= max_sequence_len:
        token_list = token_list[-(max_sequence_len-1):]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding="pre")
    predicted = model.predict(token_list, verbose=0)
    predicted_word_index = np.argmax(predicted, axis=1)[0]
    for word, index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word
    return None

input_text="To be or not to be"
print(f"Input text : {input_text}")

max_sequence_len=model.input_shape[1]+1

next_word=predict_new_word(model, tokenizer,input_text, max_sequence_len)
print(f"Next Word Prediction :{next_word}")

## Save the model
model.save("new_word_lstm.h5")

## Save the tokenizer
import pickle
with open("tokenizer.pickle", "wb") as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


Input text : To be or not to be
Next Word Prediction :the
